In [1]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from modules.utils.visualization import (
    plot_samples
)
from IPython.display import display
from pathlib import Path

pl.seed_everything(1234)

# ------------
# dataset path
# ------------
dataset_root_path = Path('/root/data/processed/drive_and_act/')
keypoint_2d_path = dataset_root_path / 'keypoint_detection_results'
keypoint_3d_path = dataset_root_path / 'annotations'
# ------------
# model
# ------------
image_width = 1280
image_height = 1024
batch_size = 64
max_epoch = 200
val_check_period = 5
early_stopping_patience = 5

Global seed set to 1234


In [2]:
import itertools
from pprint import pprint
import random
all_actors = ['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8', 'vp9', 'vp10']
combinations = list(itertools.combinations(all_actors, 2))
actor_samples = random.sample(combinations, 10)
pd.DataFrame(actor_samples)

,0,1
0,vp4,vp9
1,vp1,vp9
2,vp1,vp2
3,vp1,vp7
4,vp6,vp9
5,vp1,vp4
6,vp7,vp10
7,vp1,vp8
8,vp3,vp9
9,vp2,vp9


In [3]:
all_mpjpe = []
all_pjpe = []
all_activity_mpjpe = []


for actors in actor_samples:
    saved_model_path = f'./saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_{"_".join(sorted(actors))}'
    train_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=actors,
        exclude_ankle=True,
        exclude_hip=True
    )
    val_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_val.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_val.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=['vp11', 'vp12'],
        exclude_ankle=True,
        exclude_hip=True
    )
    test_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_test.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_test.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=['vp13', 'vp14', 'vp15'],
        exclude_ankle=True,
        exclude_hip=True
    )
    all_activities = train_dataset.activities.union(val_dataset.activities).union(test_dataset.activities)
    lit_model = LitSimpleBaselineLinear(exclude_ankle=True, exclude_hip=True, all_activities=all_activities, is_silence=True)
    print(
        'train_dataset', len(train_dataset),
        'val_dataset', len(val_dataset),
        'test_dataset', len(test_dataset)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=24)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True, num_workers=24)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=24)

    model_checkpoint = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=1)
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=early_stopping_patience)

    if not os.path.exists(saved_model_path):
        os.makedirs(saved_model_path)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    trainer = pl.Trainer(
        # max_steps=10,
        max_epochs=max_epoch,
        callbacks=[model_checkpoint, early_stopping],
        accelerator=device,
        check_val_every_n_epoch=val_check_period,
        default_root_dir=saved_model_path,
        gradient_clip_val=1.0,
        enable_progress_bar = False
    )
    trainer.fit(lit_model, train_loader, val_loader)
    with open(f'{saved_model_path}/best_model_path.txt', 'w') as f:
        f.writelines(model_checkpoint.best_model_path)
    best_checkpoint_path = model_checkpoint.best_model_path
    trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)
    all_mpjpe.append(lit_model.test_history[0]['mpjpe'])
    all_pjpe.append(lit_model.test_history[0]['pjpe'])
    all_activity_mpjpe.append(lit_model.test_history[0]['activities_mpjpe'])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp4_vp9/lightning_logs


train_dataset 4967 val_dataset 5103 test_dataset 8414


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/opt/conda/lib/python3.10/shutil.py", line 731, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/opt/conda/lib/python3.10/shutil.py", line 729, in rmtree
    os.rmdir(path)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-n0sm9l51'
Restoring states from the checkp

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │     81.26601387879039     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp1_vp9/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/prediction/linear_model_single_actor/actor_vp6_vp7/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)


train_dataset 5322 val_dataset 5103 test_dataset 8414


Exception ignored in: <function _releaseLock at 0x7fc111e22f80>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


: 

: 

In [ ]:
pd.DataFrame(all_mpjpe, index=actor_samples)

In [ ]:
pd.DataFrame(all_mpjpe, index=all_actors).mean()

In [ ]:
pjpe_df = pd.concat(all_pjpe, axis=1)
pjpe_df.columns = all_actors
pjpe_df

In [ ]:
pd.DataFrame(pjpe_df.mean(axis=1), columns=['average_pjpe'])

In [ ]:
pd.DataFrame(all_activity_mpjpe, index=all_actors).T

In [ ]:
pd.DataFrame(pd.DataFrame(all_activity_mpjpe, index=all_actors).T.mean(axis=1), columns=['pjpe'])